In [1]:
import pandas as pd
import numpy as np
import json
from pyearth import Earth
import plotly.graph_objects as go
import datetime as dt
from re import findall
from sklearn.preprocessing import MinMaxScaler

In [81]:
f = open('df_checked_dict.json')
data = json.load(f)

In [82]:
df_shib = pd.DataFrame.from_dict(data['shib']).dropna()
df_shib['snapped_at'] = pd.to_datetime(df_shib['snapped_at'], format='%Y-%m-%d %H:%M:%S UTC')
df_shib = adjust_dt(df_shib,1)

In [78]:
PCT_THRESHOLD = 0.2
REGION_THRESHOLD = 30
df_shib_up = df_shib[np.abs(df_shib['pct_change']) > PCT_THRESHOLD]
df_shib_up['region_length'] = df_shib_up['snapped_at'].diff()
df_shib_region = df_shib_up[df_shib_up['region_length'] >= pd.Timedelta(REGION_THRESHOLD,'d')]

c:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [83]:
df_shib

,snapped_at,price,market_cap,total_volume,pct_change
1,2020-08-02,9.722410e-10,0.000000e+00,1.426169e+06,0.000000
2,2020-08-03,5.169502e-10,0.000000e+00,1.773209e+05,-0.468290
3,2020-08-04,1.471351e-10,0.000000e+00,5.897767e+04,-0.715379
4,2020-08-05,1.635567e-10,0.000000e+00,1.892398e+04,0.111609
5,2020-08-06,3.161971e-10,0.000000e+00,3.738508e+04,0.933256
...,...,...,...,...,...
681,2022-06-27,1.105079e-05,6.544687e+09,6.776310e+08,-0.063705
682,2022-06-28,1.106403e-05,6.510723e+09,5.404461e+08,0.001198
683,2022-06-29,9.981285e-06,5.883186e+09,4.774144e+08,-0.097862
684,2022-06-30,1.017580e-05,6.004611e+09,6.142981e+08,0.019488


In [79]:
df_shib_region

,snapped_at,price,market_cap,total_volume,pct_change,region_length
169,2021-01-31,5.053100e-09,0.000000e+00,1.098978e+07,34.374242,119 days
358,2021-08-08,7.789172e-06,3.846338e+09,7.145166e+08,0.234385,47 days
398,2021-09-17,8.721695e-06,4.322177e+09,1.468540e+09,0.287194,40 days
541,2022-02-07,2.805471e-05,1.520031e+10,1.975345e+09,0.244301,93 days
606,2022-04-13,2.713345e-05,1.483570e+10,4.285465e+09,0.227009,65 days
676,2022-06-22,1.058620e-05,6.185396e+09,1.839091e+09,0.297577,41 days


In [62]:
is_staircase = False
staircase = {'date_before': [],
            'date_after': []}
for j in range(len(df_shib_region)):
    date_init = df_shib_region['snapped_at'][j] - df_shib_region['region_length'][j]
    date_final = df_shib_region['snapped_at'][j]
    df_shib_filter = df_shib[np.logical_and((df_shib['snapped_at'] >= date_init),(df_shib['snapped_at'] < date_final))]
    x = df_shib_filter['snapped_at'].map(dt.datetime.toordinal)
    x = x - x[0]
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x.values.reshape(-1,1))
    y = df_shib_filter['price']
    model = Earth()
    model.fit(x,y)
    y_pred = model.predict(x)

    sum_li = model.summary().split('\n')[4:-2]
    arr_knot = []
    for i in range(len(sum_li)):
        line = sum_li[i]
        if 'Yes' not in line:
            if 'x0' in line:
                columns = line.split()
                try:
                    f = findall(r'\((.+)\)',columns[0])[0]
                    const = findall(r'[+-]?[0-9]\.[0-9]+',f)
                    const = float(const[0])
                    knot = abs(round(scaler.inverse_transform([[const]])[0][0]))
                    if knot < len(x):
                        arr_knot.append(knot)
                except Exception as e: 
                    continue
    arr_knot = np.unique(arr_knot)
    arr_grad = []
    arr_date = [date_init]
    for i in range(len(arr_knot)+1):
        if i == 0:
            x0 = 0
        else:
            x0 = int(arr_knot[i-1])

        if i == len(arr_knot):
            x1 = len(y_pred)-1
        else:
            x1 = int(arr_knot[i])
        y0 = y_pred[x0]
        y1 = y_pred[x1]
        grad = (y1-y0)/(x1-x0)
        arr_grad.append(grad)
        arr_date.append(date_init + pd.Timedelta(x1,'d'))
    for i in range(len(arr_grad)-1):
        grad_before = arr_grad[i]
        grad_after = arr_grad[i+1]
        date_before = arr_date[i]
        date_after = arr_date[i+2]
        delta_grad = (abs(grad_after)-abs(grad_before))
        if grad_before>0 and grad_before*grad_after < 0 and delta_grad/abs(grad_before) < 0.5:
            is_staircase = True
            staircase['date_before'].append(date_before)
            staircase['date_after'].append(date_after)

c:\Users\asus\Anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

c:\Users\asus\Anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

c:\Users\asus\Anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep

In [64]:
arr_knot

array([25.])

In [54]:
date_init = df_shib_region['snapped_at'][1] - df_shib_region['region_length'][1]
date_final = df_shib_region['snapped_at'][1]
df_shib_filter = df_shib[np.logical_and((df_shib['snapped_at'] >= date_init),(df_shib['snapped_at'] < date_final))]

In [55]:
x = df_shib_filter['snapped_at'].map(dt.datetime.toordinal)
x = x - x[0]
scaler = MinMaxScaler()
x = scaler.fit_transform(x.values.reshape(-1,1))
y = df_shib_filter['price']

In [56]:
model = Earth()
model.fit(x,y)
y_pred = model.predict(x)

sum_li = model.summary().split('\n')[4:-2]
arr_knot = []
for i in range(len(sum_li)):
    line = sum_li[i]
    if 'Yes' not in line:
        if 'x0' in line:
            columns = line.split()
            try:
                f = findall(r'\((.+)\)',columns[0])[0]
                const = findall(r'[+-]?[0-9]\.[0-9]+',f)
                const = float(const[0])
                arr_knot.append(abs(round(scaler.inverse_transform([[const]])[0][0])))
            except Exception as e: 
                print(e)
                continue
arr_knot = np.unique(arr_knot)

list index out of range


c:\Users\asus\Anaconda3\lib\site-packages\pyearth\earth.py:813: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

c:\Users\asus\Anaconda3\lib\site-packages\pyearth\earth.py:1066: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



In [57]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_shib_filter['snapped_at'],y=y,name='price',mode='markers'))
fig.add_trace(go.Scatter(x=df_shib_filter['snapped_at'],y=y_pred,name='trend',mode='lines'))
fig.show()

In [35]:
FAKE_PUMP_THRESHOLD = 4
FAKE_PUMP_BOUNDARY = 1
staircase['fake_pump'] = [0 for i in range(len(staircase['date_before']))]
last_day = df_shib['snapped_at'][-1]
for i in range(len(staircase['date_before'])):
    date_after = staircase['date_after'][i]
    is_search = True
    date_now = date_after
    data_avg = df_shib[df_shib['snapped_at'] <= date_after]['pct_change'].values[-2:].tolist()
    data_avg = [abs(d) for d in data_avg]
    is_fake_pump = False
    while is_search:
        temp_df = df_shib[df_shib['snapped_at'] >= date_now]
        temp_df = temp_df[temp_df['snapped_at'] <= date_now+pd.Timedelta(1,'d')]
        p0 = temp_df['pct_change'].values[0]
        p1 = temp_df['pct_change'].values[1]
        avg = np.average(data_avg)
        if p0*p1 <= 0:
            if abs(p0) > FAKE_PUMP_THRESHOLD*avg and abs(p1) > FAKE_PUMP_THRESHOLD*avg and abs(p0) < FAKE_PUMP_BOUNDARY and abs(p1) < FAKE_PUMP_BOUNDARY:
                is_fake_pump = True
                is_search = False
                staircase['fake_pump'][i] = temp_df['snapped_at'][0]
        if not is_fake_pump:
            data_avg.append(abs(p0))
        date_now = date_now+pd.Timedelta(1,'d')
        if i != len(staircase['date_before'])-1:
            if date_now >= staircase['date_before'][i+1]:
                is_search = False
        if date_now == last_day:
            is_search = False
        if abs(p0) > 0.5:
            is_search = False

In [47]:
def adjust_dt(df, delta_time):
    date_init =  df['snapped_at'][0]
    idx_date = df['snapped_at'].map(dt.datetime.toordinal)
    idx_date = idx_date - idx_date[0]
    df_modify = df[df['snapped_at'] == date_init]
    for i in idx_date:
        if i!=0:
            if i%delta_time == 0:
                df_modify = pd.concat([df_modify,df[df['snapped_at'] == date_init+pd.Timedelta(i,'d')]])
    return df_modify


In [58]:
arr_knot

array([], dtype=float64)

In [42]:
df_shib['snapped_at'].map(dt.datetime.toordinal)

1      737639
2      737640
3      737641
4      737642
5      737643
        ...  
681    738333
682    738334
683    738335
684    738336
685    738337
Name: snapped_at, Length: 685, dtype: int64

In [75]:
temp_df['price'][1] = 1

c:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
temp_df

,snapped_at,price,market_cap,total_volume,pct_change
684,2022-06-30,1.0,6.004611e+09,6.142981e+08,0.019488
685,2022-07-01,1.0,5.967997e+09,5.995007e+08,0.000554
